# Library

In [2]:
import torch
import torch.nn as nn
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torch.autograd import Variable

# Congfig

In [19]:
# konfigurasi
input_size = 784 # 28 x 28 (panjang x lebar)
hidden_size = 400 # jumlah neuron pada hidden layer
out_size = 10 # jumlah neuron pada output layer (0-9 : 10 digit)
batch_size = 100 # banyak sampel yang akan diproses dalam satu iterasi
epochs = 10
learning_rate = 0.001 # LR untuk optimizer

# Dataset

In [4]:
# unduh dataset MNIST
train_dataset = datasets.MNIST(root='./data', train=True, transform=transforms.ToTensor(), download=True)

test_dataset = datasets.MNIST(root='./data', train=False, transform=transforms.ToTensor())

In [5]:
# data loader
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size,
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=batch_size,
                                          shuffle=False)

In [ ]:
# lihat train loader
# for i, (images, labels) in enumerate(train_loader):
    # print(images.size())
    # images = images.view(-1, 28 * 28)  # ubah ukuran gambar menjadi vektor 1D
    # print(images.size())

    # output : (batch_size, 784)

In [ ]:
# melihat data pada data loader
print("Ada {} batch di dataset".format(len(train_loader)))

for (x, y) in train_loader:
    print("Untuk satu iterasi (batch) ada :")
    print("Data : {}".format(x.shape))
    print("Label : {}".format(y.shape))
    break

# Model

In [12]:
class Net(nn.Module):
    def __init__(self, input_size, hidden_size, out_size):
        '''
        Inisialisasi arstektur model 
        Args :
            input_size : ukuran fitur atau innput
            hidden_size : ukuran neuron pada hidden layer
            out_size : ukuran neuron untuk output (banyak kelas label/target)

            fc1, fc2, fc3 : tiga lapisan fully connected
            relu : aktivasi relu
        '''
        super(Net, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, hidden_size)
        self.fc3 = nn.Linear(hidden_size, out_size)
        self.relu = nn.ReLU()

    def forward(self, x):
        '''
        Proses forward
        '''
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        out = self.relu(out)
        out = self.fc3(out)
            
        return out

In [13]:
net = Net(input_size, hidden_size, out_size)

CUDA = torch.cuda.is_available()

if CUDA:                # mengembalikan boolean
    net = net.cuda()

In [14]:
# loss function
criterion = nn.CrossEntropyLoss() # comes along with sofmax

# optimizer
optimizer = torch.optim.Adam(net.parameters(), lr=learning_rate)

# Training loop

In [21]:
correct_train = 0
total_train = 0

for epoch in range(epochs):
    for i, (images, labels) in enumerate(train_loader):
        # ubak ukuran gambar menjadi vektor 1D (batch, 1, 28, 28) -> (batch, 784)
        # Wrap ke variable
        images = Variable(images.view(-1, 28 * 28))
        labels = Variable(labels)

        # pindahkan ke GPU
        if CUDA:
            images = images.cuda()
            labels = labels.cuda()

        # kembalikan gradien ke nol
        optimizer.zero_grad()

        # forward
        outputs = net(images)

        # dapatkan indeks dari output dengan probabilitas tertinggi
        _, predicted = torch.max(outputs.data, 1) # _ : nilai maksimum, predicted : indeks dari nilai maksimum tersebut

        total_train += labels.size(0)

        if CUDA:
            correct_train += (predicted.cpu() == labels.cpu()).sum() # sum() tidak ada di GPU, jadi pindahkan ke CPU
        else:
            correct_train += (predicted == labels).sum()

        # hitung loss
        loss = criterion(outputs, labels)

        # backward
        loss.backward()

        # update parameter
        optimizer.step()    

        if (i + 1) % 100 == 0:
            print("Epoch {}/{} | Iterasi {}/{} | Loss Training : {:.2f} | Akurasi Training :  {:.2f}%".format(
            epoch + 1, epochs, i + 1, len(train_dataset)//batch_size, loss.item(), (100 * correct_train / total_train)))

print("Training Selesai!")

Epoch 1/10 | Iterasi 100/600 | Loss Training : 0.01 | Akurasi Training :  99.61%
Epoch 1/10 | Iterasi 200/600 | Loss Training : 0.05 | Akurasi Training :  99.60%
Epoch 1/10 | Iterasi 300/600 | Loss Training : 0.07 | Akurasi Training :  99.53%
Epoch 1/10 | Iterasi 400/600 | Loss Training : 0.03 | Akurasi Training :  99.46%
Epoch 1/10 | Iterasi 500/600 | Loss Training : 0.03 | Akurasi Training :  99.47%
Epoch 1/10 | Iterasi 600/600 | Loss Training : 0.00 | Akurasi Training :  99.48%
Epoch 2/10 | Iterasi 100/600 | Loss Training : 0.00 | Akurasi Training :  99.52%
Epoch 2/10 | Iterasi 200/600 | Loss Training : 0.05 | Akurasi Training :  99.54%
Epoch 2/10 | Iterasi 300/600 | Loss Training : 0.00 | Akurasi Training :  99.55%
Epoch 2/10 | Iterasi 400/600 | Loss Training : 0.00 | Akurasi Training :  99.53%
Epoch 2/10 | Iterasi 500/600 | Loss Training : 0.02 | Akurasi Training :  99.51%
Epoch 2/10 | Iterasi 600/600 | Loss Training : 0.03 | Akurasi Training :  99.50%
Epoch 3/10 | Iterasi 100/600

# Testing

**Pada saat testing, tidak menghitung loss dan bobot serta tidak ada update parameter**

In [25]:
correct = 0
total = 0

for images, labels in test_loader:
    # ubah ukuran gambar menjadi vektor 1D
    images = Variable(images.view(-1, 28 * 28))

    if CUDA:
        images = images.cuda()

    # untuk setiap input dalam batch, akan memberikan output 10 elemen
    outputs = net(images)

    # dapatkan indeks probabilitas tertinggi
    _, predicted = torch.max(outputs.data, 1)

    total += labels.size(0)

    if CUDA:
        correct += (predicted.cpu() == labels.cpu()).sum()
    else:
        correct += (predicted == labels).sum()


print("Final Akurasi Test : {:.2f}%".format(100 * correct / total))

Final Akurasi Test : 98.01%
